In [1]:
import pandas as pd
import numpy as np

# Load the data safely
file_path = 'Airbnb_Open_Data.csv'

# Read the CSV while skipping bad lines
df = pd.read_csv(file_path, low_memory=False, on_bad_lines='skip')

# 1. Clean column names properly: strip, lowercase, add underscores
df.columns = (df.columns
                .str.strip()
                .str.lower()
                .str.replace(' ', '_')   # REPLACE SPACES with underscores
                .str.replace('-', '_'))   # if any dashes also
              
# 1. Strip whitespace from column names
df.columns = df.columns.str.strip()

# 2. Drop completely empty columns
df = df.dropna(axis=1, how='all')

# 3. Drop duplicate rows
df = df.drop_duplicates()

# 4. Remove junk columns like "Unnamed: 0"
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 5. Clean text columns properly with 'string' dtype
text_columns = df.select_dtypes(include='object').columns
for col in text_columns:
    df[col] = df[col].astype('string').str.strip()    # real string type
    df[col] = df[col].replace('', pd.NA)               # empty strings to real missing

# 6. Fix price-related columns to float
price_cols = ['price', 'service_fee', 'cleaning_fee', 'security_deposit', 'extra_people']
for col in price_cols:
    if col in df.columns:
        df[col] = (df[col]
                     .replace(r'[\$,]', '', regex=True)
                     .replace('', pd.NA)
                     .astype(float))

# 7. Force important columns to 'string'
string_cols = ['country', 'state', 'city', 'street', 'name', 'host_name']
for col in string_cols:
    if col in df.columns:
        df[col] = df[col].astype('string').str.strip()
        df[col] = df[col].replace('', pd.NA)

# 8. Standardize real date columns
# First find all columns that have 'date' in name
if 'last_review' in  df.columns:
    df['last_review'] = pd.to_datetime(df['last_review'], errors = 'coerce')

# 9. Reset index
df = df.reset_index(drop=True)


In [18]:
df.head()

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,193.0,10.0,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,<NA>
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,28.0,30.0,45.0,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,<NA>
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,<NA>,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,124.0,3.0,0.0,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",<NA>
3,1002755,<NA>,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,74.0,30.0,270.0,2019-07-05,4.64,4.0,1.0,322.0,<NA>,<NA>
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,41.0,10.0,9.0,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",<NA>


In [20]:
# what is average price per neighbourhood group 

In [22]:
df.groupby('neighbourhood_group')['price'].mean()

neighbourhood_group
Bronx            627.764793
Brooklyn         626.501962
Manhattan        622.650931
Queens           630.217031
Staten Island    623.990486
brookln          580.000000
manhatan         460.000000
Name: price, dtype: float64

In [24]:
df.groupby('neighbourhood_group').mean()

TypeError: agg function failed [how->mean,dtype->string]

In [26]:
# count lisiting per room

In [28]:
df.groupby('room_type').size()

room_type
Entire home/apt    53429
Hotel room           115
Private room       46306
Shared room         2208
dtype: int64

In [34]:
df.groupby('neighbourhood_group')[['price','availability_365']].agg(['mean','max'])

price         availability_365        
                           mean     max             mean     max
neighbourhood_group                                             
Bronx                627.764793  1200.0       179.077670   426.0
Brooklyn             626.501962  1200.0       129.746290   426.0
Manhattan            622.650931  1200.0       141.975663  3677.0
Queens               630.217031  1200.0       161.701721   426.0
Staten Island        623.990486  1200.0       197.451477   425.0
brookln              580.000000   580.0        47.000000    47.0
manhatan             460.000000   460.0       325.000000   325.0

In [38]:
#give me sum of reviews and avg price and service fee by room type
df.groupby('room_type')[['number_of_reviews','price','service_fee']].agg(['sum','mean'])

number_of_reviews                  price              \
                              sum       mean         sum        mean   
room_type                                                              
Entire home/apt         1462889.0  27.418544  33322794.0  625.240056   
Hotel room                 9669.0  84.078261     76635.0  666.391304   
Private room            1285994.0  27.836573  28872026.0  624.935628   
Shared room               44839.0  20.307518   1396622.0  634.828182   

                service_fee              
                        sum        mean  
room_type                                
Entire home/apt   6662761.0  125.000206  
Hotel room          14949.0  132.292035  
Private room      5769449.0  124.971819  
Shared room        279961.0  127.024047

In [52]:
# average price per neighbourhood and room type

pd.pivot_table(df, values = 'price', index = 'neighbourhood_group', columns = 'room_type', aggfunc = 'mean')

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood_group,,,,
Bronx,1017.0,NaN,1555.0,115.0
Brooklyn,20432.0,7.0,20279.0,819.0
Manhattan,26259.0,100.0,16196.0,892.0
Queens,5105.0,8.0,7692.0,359.0
Staten Island,473.0,NaN,458.0,15.0
brookln,NaN,NaN,1.0,NaN
manhatan,NaN,NaN,1.0,NaN


In [46]:
pd.pivot_table(df, values = 'price', index = 'neighbourhood_group', columns = 'room_type', aggfunc = 'count')

room_type,Entire home/apt,Hotel room,Private room,Shared room
neighbourhood_group,,,,
Bronx,1017.0,NaN,1555.0,115.0
Brooklyn,20432.0,7.0,20279.0,819.0
Manhattan,26259.0,100.0,16196.0,892.0
Queens,5105.0,8.0,7692.0,359.0
Staten Island,473.0,NaN,458.0,15.0
brookln,NaN,NaN,1.0,NaN
manhatan,NaN,NaN,1.0,NaN


In [56]:
pd.pivot_table(df, values = 'availability_365', index = 'room_type', aggfunc = 'count')

,availability_365
room_type,
Entire home/apt,53216
Hotel room,115
Private room,46086
Shared room,2193


In [98]:
# create a pivot table showing sum of reviews per host name and country
piv = pd.pivot_table(df,values = 'number_of_reviews', index = 'host_name', columns ='country' , aggfunc = 'sum', sort = True)

In [102]:
piv['total'] = piv.sum(axis = 1)

In [108]:
df.columns

Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'license'],
      dtype='object')

In [110]:
piv.sort_values(by ='total', ascending=False)

country,United States,total
host_name,,
Michael,24488.0,24488.0
David,17621.0,17621.0
John,15538.0,15538.0
Alex,13891.0,13891.0
Eric,12095.0,12095.0
...,...,...
Geoffray,0.0,0.0
Lorrayne,0.0,0.0
Genya,0.0,0.0


In [112]:
 df[['host_id','host_name','host_identity_verified']]

,host_id,host_name,host_identity_verified
0,80014485718,Madaline,unconfirmed
1,52335172823,Jenna,verified
2,78829239556,Elise,<NA>
3,85098326012,Garry,unconfirmed
4,92037596077,Lyndon,verified
...,...,...,...
102053,77326652202,Mariam,unconfirmed
102054,45936254757,Trey,verified
102055,23801060917,Michael,verified
102056,15593031571,Shireen,unconfirmed


In [114]:
hosts = df[['host_id','host_name','host_identity_verified']].drop_duplicates()

In [116]:
hosts

,host_id,host_name,host_identity_verified
0,80014485718,Madaline,unconfirmed
1,52335172823,Jenna,verified
2,78829239556,Elise,<NA>
3,85098326012,Garry,unconfirmed
4,92037596077,Lyndon,verified
...,...,...,...
102053,77326652202,Mariam,unconfirmed
102054,45936254757,Trey,verified
102055,23801060917,Michael,verified
102056,15593031571,Shireen,unconfirmed


In [118]:
listings = df[['id','host_id','name','price','neighbourhood_group','room_type']]

In [120]:
listings

,id,host_id,name,price,neighbourhood_group,room_type
0,1001254,80014485718,Clean & quiet apt home by the park,966.0,Brooklyn,Private room
1,1002102,52335172823,Skylit Midtown Castle,142.0,Manhattan,Entire home/apt
2,1002403,78829239556,THE VILLAGE OF HARLEM....NEW YORK !,620.0,Manhattan,Private room
3,1002755,85098326012,<NA>,368.0,Brooklyn,Entire home/apt
4,1003689,92037596077,Entire Apt: Spacious Studio/Loft by central park,204.0,Manhattan,Entire home/apt
...,...,...,...,...,...,...
102053,57365208,77326652202,Cozy bright room near Prospect Park,696.0,Brooklyn,Private room
102054,57365760,45936254757,Private Bedroom with Amazing Rooftop View,909.0,Brooklyn,Private room
102055,57366313,23801060917,Pretty Brooklyn One-Bedroom for 2 to 4 people,387.0,Brooklyn,Entire home/apt
102056,57366865,15593031571,Room & private bathroom in historic Harlem,848.0,Manhattan,Private room


In [122]:
combined_df = pd.merge(listings,hosts,on='host_id',how='left')

In [124]:
combined_df.head()

,id,host_id,name,price,neighbourhood_group,room_type,host_name,host_identity_verified
0,1001254,80014485718,Clean & quiet apt home by the park,966.0,Brooklyn,Private room,Madaline,unconfirmed
1,1002102,52335172823,Skylit Midtown Castle,142.0,Manhattan,Entire home/apt,Jenna,verified
2,1002403,78829239556,THE VILLAGE OF HARLEM....NEW YORK !,620.0,Manhattan,Private room,Elise,<NA>
3,1002755,85098326012,<NA>,368.0,Brooklyn,Entire home/apt,Garry,unconfirmed
4,1003689,92037596077,Entire Apt: Spacious Studio/Loft by central park,204.0,Manhattan,Entire home/apt,Lyndon,verified


In [126]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102058 entries, 0 to 102057
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   id                              102058 non-null  int64         
 1   name                            101808 non-null  string        
 2   host_id                         102058 non-null  int64         
 3   host_identity_verified          101769 non-null  string        
 4   host_name                       101654 non-null  string        
 5   neighbourhood_group             102029 non-null  string        
 6   neighbourhood                   102042 non-null  string        
 7   lat                             102050 non-null  float64       
 8   long                            102050 non-null  float64       
 9   country                         101526 non-null  string        
 10  country_code                    101927 non-null  string 

In [130]:
#number of reviews per month
df.set_index('last_review').resample('M')['number_of_reviews'].sum()

C:\Users\user\AppData\Local\Temp\ipykernel_41276\1104751791.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df.set_index('last_review').resample('M')['number_of_reviews'].sum()


last_review
2012-07-31      2.0
2012-08-31      4.0
2012-09-30     24.0
2012-10-31      0.0
2012-11-30      8.0
              ...  
2058-02-28      0.0
2058-03-31      0.0
2058-04-30      0.0
2058-05-31      0.0
2058-06-30    124.0
Name: number_of_reviews, Length: 552, dtype: float64

In [140]:
#adding month level data
df['last_review_month'] = df['last_review'].dt.month

In [142]:
df.head()

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license,last_review_month
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,10.0,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,<NA>,10.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,30.0,45.0,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,<NA>,5.0
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,<NA>,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,3.0,0.0,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",<NA>,NaN
3,1002755,<NA>,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,30.0,270.0,2019-07-05,4.64,4.0,1.0,322.0,<NA>,<NA>,7.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,10.0,9.0,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",<NA>,11.0


In [144]:
df['last_review_year'] = df['last_review'].dt.year

In [146]:
df.head()

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license,last_review_month,last_review_year
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,9.0,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,<NA>,10.0,2021.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,45.0,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,<NA>,5.0,2022.0
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,<NA>,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,0.0,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",<NA>,NaN,NaN
3,1002755,<NA>,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,270.0,2019-07-05,4.64,4.0,1.0,322.0,<NA>,<NA>,7.0,2019.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,9.0,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",<NA>,11.0,2018.0


In [150]:
#adding month & year level data
#df['last_review_month'] = df['last_review'].dt.month
#df['last_review_year'] = df['last_review'].dt.year

#count reviews in 2022 by month

df_2022 = df[df['last_review_year'] == 2022]

In [152]:
df_2022.groupby('last_review_month')['number_of_reviews'].sum()

last_review_month
1.0    153581.0
2.0    341899.0
3.0     45188.0
4.0         3.0
5.0        50.0
Name: number_of_reviews, dtype: float64

In [161]:
#count how many listing were in july 2021
df[(df['last_review_year'] == 2021) & (df['last_review_month'] == 7)].shape[0]

409

In [ ]:
# i want to multiple a number by 2

In [163]:
def multi(x):
    return x * 2

In [167]:
print(multi(5234532))

10469064


In [171]:
f = lambda x:x*2
print(f(5))

10


In [177]:
# to identify a number is even or odd

check_even = lambda n: 'even' if n % 2 == 0 else 'odd'

In [183]:
print(check_even(325235321135))

odd


In [185]:
#label expensive listings 
#price_category = if price is > 800 its expensive else its cheap

df['price_category'] = df['price'].apply(lambda x: 'High' if x > 800 else 'Low')

In [187]:
df

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license,last_review_month,last_review_year,price_category
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,2021-10-19,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,<NA>,10.0,2021.0,High
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,2022-05-21,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,<NA>,5.0,2022.0,Low
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,<NA>,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,NaT,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",<NA>,NaN,NaN,Low
3,1002755,<NA>,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,2019-07-05,4.64,4.0,1.0,322.0,<NA>,<NA>,7.0,2019.0,Low
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,2018-11-19,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",<NA>,11.0,2018.0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102053,57365208,Cozy bright room near Prospect Park,77326652202,unconfirmed,Mariam,Brooklyn,Flatbush,40.64945,-73.96108,United States,...,2019-03-27,0.44,5.0,1.0,0.0,Shoes off Clean After yourself Turn Lights and...,<NA>,3.0,2019.0,Low
102054,57365760,Private Bedroom with Amazing Rooftop View,45936254757,verified,Trey,Brooklyn,Bushwick,40.69872,-73.92718,United States,...,2017-08-31,0.72,3.0,2.0,0.0,#NAME?,<NA>,8.0,2017.0,High
102055,57366313,Pretty Brooklyn One-Bedroom for 2 to 4 people,23801060917,verified,Michael,Brooklyn,Bedford-Stuyvesant,40.67810,-73.90822,United States,...,2019-06-26,3.12,4.0,2.0,235.0,* Check out: 10am * We made an effort to keep ...,<NA>,6.0,2019.0,Low
102056,57366865,Room & private bathroom in historic Harlem,15593031571,unconfirmed,Shireen,Manhattan,Harlem,40.81248,-73.94317,United States,...,NaT,NaN,1.0,1.0,0.0,Each of us is working and/or going to school a...,<NA>,NaN,NaN,High


In [189]:
#def price_category(x):
   # if['price'] > 800:
    #somethihng
   # else
    #something
   # return #something
#df[price_cateogy] = df[price].apply(price_category)

In [195]:
df.describe()

,id,host_id,lat,long,construction_year,price,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,last_review_month,last_review_year
count,1.020580e+05,1.020580e+05,102050.000000,102050.000000,101844.000000,101811.000000,101785.000000,101658.000000,101875.000000,86226,86240.000000,101739.000000,101739.000000,101610.000000,86226.000000,86226.000000
mean,2.918438e+07,4.926738e+10,40.728097,-73.949652,2012.488090,625.355580,125.039249,8.126640,27.517948,2019-06-11 02:12:22.996312064,1.375408,3.278733,7.936937,141.043992,5.889279,2018.995059
min,1.001254e+06,1.236005e+08,40.499790,-74.249840,2003.000000,50.000000,10.000000,-1223.000000,0.000000,2012-07-11 00:00:00,0.010000,1.000000,1.000000,-10.000000,1.000000,2012.000000
25%,1.509286e+07,2.459924e+10,40.688730,-73.982580,2007.000000,340.000000,68.000000,2.000000,1.000000,2018-10-27 00:00:00,0.220000,2.000000,1.000000,3.000000,4.000000,2018.000000
50%,2.918438e+07,4.912865e+10,40.722290,-73.954440,2012.000000,625.000000,125.000000,3.000000,7.000000,2019-06-13 00:00:00,0.740000,3.000000,1.000000,96.000000,6.000000,2019.000000
75%,4.327590e+07,7.400619e+10,40.762760,-73.932350,2017.000000,913.000000,183.000000,5.000000,31.000000,2019-07-05 00:00:00,2.010000,4.000000,2.000000,268.000000,7.000000,2019.000000
max,5.736742e+07,9.876313e+10,40.916970,-73.705220,2022.000000,1200.000000,240.000000,5645.000000,1024.000000,2058-06-16 00:00:00,90.000000,5.000000,332.000000,3677.000000,12.000000,2058.000000
std,1.627173e+07,2.853742e+10,0.055852,0.049502,5.765838,331.672649,66.325905,30.616698,49.571744,NaN,1.748019,1.285089,32.266355,135.429156,3.020256,1.668621


In [197]:
#add a new column saying  lisiting as newbie if host has <= 10 review else 'super host' host_category
#df['price_category'] = df['price'].apply(lambda x: 'High' if x > 800 else 'Low')
df['host_category'] = df['number_of_reviews'].apply(lambda x: 'Newbie' if x<= 10 else 'super host')

In [199]:
df

,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license,last_review_month,last_review_year,price_category,host_category
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,<NA>,10.0,2021.0,High,Newbie
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,<NA>,5.0,2022.0,Low,super host
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,<NA>,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",<NA>,NaN,NaN,Low,Newbie
3,1002755,<NA>,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,4.64,4.0,1.0,322.0,<NA>,<NA>,7.0,2019.0,Low,super host
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",<NA>,11.0,2018.0,Low,Newbie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102053,57365208,Cozy bright room near Prospect Park,77326652202,unconfirmed,Mariam,Brooklyn,Flatbush,40.64945,-73.96108,United States,...,0.44,5.0,1.0,0.0,Shoes off Clean After yourself Turn Lights and...,<NA>,3.0,2019.0,Low,super host
102054,57365760,Private Bedroom with Amazing Rooftop View,45936254757,verified,Trey,Brooklyn,Bushwick,40.69872,-73.92718,United States,...,0.72,3.0,2.0,0.0,#NAME?,<NA>,8.0,2017.0,High,super host
102055,57366313,Pretty Brooklyn One-Bedroom for 2 to 4 people,23801060917,verified,Michael,Brooklyn,Bedford-Stuyvesant,40.67810,-73.90822,United States,...,3.12,4.0,2.0,235.0,* Check out: 10am * We made an effort to keep ...,<NA>,6.0,2019.0,Low,super host
102056,57366865,Room & private bathroom in historic Harlem,15593031571,unconfirmed,Shireen,Manhattan,Harlem,40.81248,-73.94317,United States,...,NaN,1.0,1.0,0.0,Each of us is working and/or going to school a...,<NA>,NaN,NaN,High,Newbie
